<style>
div.warn{
    display: all;
    position: relative;
    justify-content: center;
    align-items: center;
    color: black;
    align: center;
    font-family: Garamond;
    font-size: 18px;
    margin: 0em 0em 0em 0em;/* top, right, bottom, left */
    width: 1000px;
    height: 160px;
    padding: 0.5em;
    text-align: center;
    text-indent: 0px;
    text-shadow: 0px;
    border-width:5px;
    border-color:#ffffff;/*white;*/
    border-style: double;
    border-radius: 2px;
    background-color: #00b8ff;
}

</style>
<div class=warn>
<img src="./data/maugustomaidana.jpg" alt="Augusto Maidana" align="left" width="120px" height="120px" >
<img src="./data/it_academy_logo.png" alt="IT Academy" align="bottom" width="30%" height="30%" ><br>
<h1><b>S11 T01:</b> Practicant amb training i test sets</h1>

<br>

<p style="text-align:left;">
    <b>M. Augusto Maidana Silanes</b>
    <span style="float:right;">
        13/10/2021
    </span>
</p>
</div>

<br>

## Descripció

Familiaritza't amb la programació científica mitjantçant la llibreria SKLearn / Scikitlearn.

**Nivell 1**

- Exercici 1:

Parteix el conjunt de dades ``DelayedFlights.csv`` en train i test. Estudia els dos conjunts per separat, a nivell descriptiu.

In [ ]:
# Render our plots inline
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, power_transform, quantile_transform, PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
#from pandas import DataFrame

In [ ]:
#plt.show(block=True)
#pd.describe_option()
plt.style.use('ggplot')
#plt.style.use('default')
plt.rcParams['figure.figsize'] = (15, 5)

directory = "../estructures_dataframe/"
file = "DelayedFlights.csv"
# Set floating-point format
pd.set_option('float_format', '{:.2f}'.format)
# Reading data from CSV: with CSV files a single line is needed to load the data:
DelayedFlights_df = pd.read_csv(directory + file, index_col=False)
DelayedFlights_df.reset_index(drop=True, inplace=True)
DelayedFlights_df.drop('Unnamed: 0', axis=1, inplace=True)

# let's see the DataFrame
DelayedFlights_df.head()

### We are going to print the number of rows and columns of the data frame:

In [ ]:
# Data Frame shape
DelayedFlights_df.shape

### The following is a summary of some basic statistical details:

In [ ]:
# print a statistical description of the data set
DelayedFlights_df.describe(include='all')

### Let's look at the names of the columns of the data frame:

In [ ]:
# Data Frame columns
DelayedFlights_df.columns

In [ ]:
# rename the "UniqueCarrier" column to "AL"
DelayedFlights_df.rename({'UniqueCarrier':'AL'}, inplace=True,axis=1)
DelayedFlights_df.head()

### We select the columns we are going to work on and print the first 5 rows of the DataFrame

#### Brief description of the columns and their types and units:

**DepTime** :
Actual Departure Time (stored as float, local time in hhmm)

**ArrTime** :
Actual Arrival Time (stored as float, local time in hhmm)

**AirTime** :
Airborne Time for the flight, in minutes (stored as integer).

**ArrDelay** :
Arrival Delay, in minutes (stored as integer).

**DepDelay** :
Departure Delay, in minutes (stored as integer).

**Distance** :
Flight Distance between airports (stored as integer, distance in miles = 1,60934 km)

**CarrierDelay** :
Delay, in minutes, attributable to the carrier (stored integer).

**Note**: DepTime and ArrTime are the real times at which take-off and landing took place. The hours of the take-off and landing is coded as a float where the two first digits indicate the hour and the two last, the minutes.

In [ ]:
# selection of the set of columns to work with
subset_df = DelayedFlights_df[['AL', 'DepTime', 'ArrTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Distance', 'CarrierDelay']].copy()
subset_df.head()

### Let's print a concise summary of a DataFrame:

In [ ]:
# print non-null values
subset_df.iloc[: 10000].info(verbose=True)

In [ ]:
# print null values
subset_df.isnull().sum()

### Filtering to check for rows containing NANs in the dataset:

In [ ]:
# creating bool series True for NaN values
mask = subset_df.loc[:,['ArrTime', 'AirTime', 'ArrDelay', 'CarrierDelay']].isna().any(axis=1)

# filtering data: displaying data only with values = NaN 
subset_df[mask]

### We eliminate the columns that have NAN:

In [ ]:
# droping the rows whose all data is missing or contain null values (NaN)
subset_df = subset_df.dropna(axis = 0, how ='any')

### Check if the NANs still exist in the data set:

In [ ]:
subset_df.info()

In [ ]:
# plot the histogram of the numerical data
subset_df.select_dtypes(exclude='object').hist(figsize=(20, 12), color='green', bins=24)
plt.show()

### Two flight frequency peaks are observed between 15-20 for takeoff and 20-24 hours for landing, DepTime and ArrTime respectively:

**Note**: using bins = 24 we represent one bar per hour  

In [ ]:
# illustrate histogram details of Departure and Arrival Times
features = ['DepTime', 'ArrTime']
subset_df[features].hist(figsize=(14, 5), color='green', bins=24)

### When calculating a kernel density estimate, a sawing effect is produced by the way the data is represented. By representing the hour with the first two digits and the minutes with the next two digits, we only have flights in the first 60 digits of the 100 with which the hour is represented:

In [ ]:
# illustrate histogram details of Departure Times
fig, ax = plt.subplots(figsize=(11, 9))
sns.histplot(data=subset_df, x="DepTime", stat='frequency', kde=True, common_norm=True, log_scale=False, element="step", fill=True, bins=24, color='green')
ax.set_xticks([0,400,800,1200,1600,2000,2400])
ax.set_xticklabels(['00:00','04:00','08:00','12:00','16:00','20:00','00:00'])
ax.set(xlabel='Departure Time (hh:mm)')
ax.set_xlim([0,2400])
ax.set_ylim([0,1250])
ax.set_yticks([0,250,500,750,1000,1250])
plt.show()

### We have calculated the correlation between the variables and found very logical relationships, such as flight time being strongly related to distance or flight delay being related to departure delay. It is also observed that departure or arrival delays condition in the same way, almost 50% of airline delays. 

In [ ]:
# adjust plot
#sns.set(rc={'figure.figsize': (14, 5)})
output="corr_matrix.png"

# Compute the correlation matrix
corr_mat = subset_df.select_dtypes(include='number').corr()

# Generate a mask for the upper triangle without the diagonal k = 0
mask = np.triu(np.ones_like(corr_mat, dtype=bool), k=0)

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)
#cmap = sns.cubehelix_palette(20, light=0.95, dark=0.15)

# Draw the heatmap with the mask and correct aspect ratio
sns_plot = sns.heatmap(corr_mat, mask=mask,  annot=True, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

# save to file
#fig = sns_plot.get_figure()
#fig.savefig(output)

### Let’s split our data into features (X) and target (y) sets:

In [ ]:
# So, let’s first split our data into training and testing sets:
# Create X features (DataFrame)
X = subset_df.loc[:,['AL', 'DepTime', 'ArrTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Distance']]
# Create y responses (Series)
y = subset_df.loc[:,'CarrierDelay']
#X = subset_df.iloc[:, :6]
#y = subset_df.iloc[:, 6:7]
print(X.head())
print(y.head())

### Split training and test data:

In [ ]:
# split a dataset into train and test sets
from sklearn.model_selection import train_test_split
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### Description of training data:

In [ ]:
X_train.describe()

### Description of test data:

In [ ]:
X_test.describe()

### Null hypothesis test for training and test samples:

In [ ]:
from scipy import stats

alpha = 0.05
null_test = {}
for item in list(X_test.select_dtypes(exclude='object').columns):
    stat, p = stats.ttest_ind(X_train[item], X_test[item], alternative= 'two-sided')
    if p > alpha:
	    null_test[item] = [stat, p, alpha, "Not-Ref"]        
    else:
	    null_test[item] = [stat, p, alpha, "Refused"]

null_test = pd.DataFrame(null_test, index=['Stat', 'p', 'alpha', 'H0']).transpose()
print('Null hypothesis test for Training and Test samples (H0 != H1):\n', null_test)

**Nivell 2**

- Exercici 2:

Aplica algun procés de transformació (estandarditzar les dades numèriques, crear columnes dummies, polinomis...).

### Obtain numerical data from the training data set:

In [ ]:
# drop non-numeric cols on training data
X_train_num = X_train._get_numeric_data() 
#X_train_num = X_train.select_dtypes(exclude='object')
X_train_num.head()

### Obtain numerical data from the testing data set:

In [ ]:
# drop non-numeric cols on test data
X_test_num = X_train._get_numeric_data() 
X_test_num.head()

### Power transformations are applied to make the data more Gaussian by means of a family of parametric and monotonic transformations:

In [ ]:
X_train_num_power = power_transform(X_train_num, standardize=True)
X_train_num_power = pd.DataFrame(X_train_num_power, columns=X_train_num.columns)
X_train_num_power.head()

### We apply a type of Quantile Transformation to transform the features to a normal distribution using the quantile information:

In [ ]:
X_train_num_quant = quantile_transform(X_train_num, output_distribution='normal', random_state=42)
X_train_num_quant = pd.DataFrame(X_train_num_quant, columns=X_train_num.columns)
X_train_num_quant.head()

### We perform a second-degree polynomial decomposition of the AirTime and Distance features:

In [ ]:
X_train_time_dist = X_train_num[['AirTime','Distance']]
polynom = PolynomialFeatures(degree=2, include_bias=False)
X_train_time_dist = pd.DataFrame(polynom.fit_transform(X_train_time_dist), columns=['AirTime(X1)', 'Distance(X2)', 'X1^2', 'X1·X2','X2^2'])
X_train_time_dist.head()

In [ ]:
# power transform the AirTime data
power = PowerTransformer(method='yeo-johnson', standardize=True)
AirTime = X_train_num.AirTime.values.reshape(-1, 1)
data_trans = power.fit_transform(AirTime)
AirTime_df = pd.DataFrame(data_trans, columns=['AirTime'])
# histogram of the transformed data
sns.displot(data=AirTime_df, legend=False, bins=50, color='green')

In [ ]:
# retrieve just the numeric input values
data = X_train_num.values[:,:]
# perform a box-cox transform of the dataset
scaler = MinMaxScaler(feature_range=(1, 2))
power = PowerTransformer(method='box-cox')
pipeline = Pipeline(steps=[('s', scaler),('p', power)])
data = pipeline.fit_transform(data)
# convert the array back to a dataframe
dataset = pd.DataFrame(data, columns = ['DepTime','ArrTime','AirTime','ArrDelay','DepDelay','Distance'])
dataset.head()
# histograms of the variables
#dataset.hist(color='green', bins=50)
#plt.show()

### Histograms of the variables:

In [ ]:
# select the columns to be plotted
cols = ['DepTime', 'ArrTime','DepDelay','ArrDelay','AirTime','Distance']

# create the figure and axes
# sharex, sharey: bool or {'none', 'all', 'row', 'col'}, default: False
fig, axes = plt.subplots(3, 2, sharex='none', sharey=True, squeeze=False, gridspec_kw={"height_ratios":(.85,.85,.85)}, figsize=(10,10))
fig.suptitle('Power transformer: Box-Cox method')
axes = axes.ravel()  # flattening the array makes indexing easier

# stat: str
# Aggregate statistic to compute in each bin.
# count: show the number of observations in each bin
# frequency: show the number of observations divided by the bin width
# probability: or proportion: normalize such that bar heights sum to 1
# percent: normalize such that bar heights sum to 100
# density: normalize such that the total area of the histogram equals 1

for col, ax in zip(cols, axes):
    sns.histplot(data=dataset[col], color='green', kde=False, stat='count', ax=ax)
    ax.set_title('Feature: {} '.format(col))
    #ax.set(xlabel=col, ylabel='')
    ax.set(xlim=(-3,3), ylim=(0, None))
    x_axis = ax.axes.get_xaxis()
    x_axis.set_label_text('foo')
    x_label = x_axis.get_label()
    x_label.set_visible(False)

#plt.legend(title='', loc='upper left', labels=['Count'], bbox_to_anchor=(1, 1))
fig.tight_layout()
plt.show()
plt.close()

## Normalization using sklearn

## What is Normalization?

**Normalization is a scaling technique in which values are shifted and rescaled so that they end up ranging between 0 and 1. It is also known as Min-Max scaling**.

Here’s the formula for normalization:

$$
X' = \frac{X - X_{min}}{X_{max} - X_{min}}
$$

Here, $X_{max}$ and $X_{min}$ are the maximum and the minimum values of the feature respectively.

* When the value of $X$ is the minimum value in the column, the numerator will be $0$, and hence $X'$ is $0$.
* On the other hand, when the value of X is the maximum value in the column, the numerator is equal to the denominator and thus the value of $X'$ is $1$.
* If the value of $X$ is between the minimum and the maximum value, then the value of $X'$ is between $0$ and $1$.

Let's try to normalizate our data.

In [ ]:
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

# copy of datasets
X_train_norm = X_train_num.copy()
X_test_norm = X_test_num.copy()

# fit scaler on training data
norm = MinMaxScaler().fit(X_train_norm)

# transform training data
X_train_norm = norm.transform(X_train_norm)
X_train_norm = pd.DataFrame(X_train_norm, columns=['DepTime', 'ArrTime', 'AirTime', 'ArrDelay', 'DepDelay','Distance'])
print("Scaled Train Data: \n\n")
print(X_train_norm.head())

# transform testing data
X_test_norm = norm.transform(X_test_norm)
X_test_norm = pd.DataFrame(X_test_norm, columns=['DepTime', 'ArrTime', 'AirTime', 'ArrDelay', 'DepDelay','Distance'])
print("\n\nScaled Test Data: \n\n")
print(X_test_norm.head())

## Standardization using sklearn

## What is Standardization?

**Standardization is another scaling technique where the values are centered around the mean with a unit standard deviation. This means that the mean of the attribute becomes zero and the resultant distribution has a unit standard deviation**.

Here’s the formula for standardization:

$$
X' = \frac{X - \mu}{\sigma}
$$

* Feature scaling: $\mu$ is the mean of the feature values and
* Feature scaling: $\sigma$ is the standard deviation of the feature values. Note that in this case, the values are not restricted to a particular range.

Let’s try to standardize our data.

In [ ]:
# data standardization with sklearn
from sklearn.preprocessing import StandardScaler

# copy of datasets
X_train_stand = X_train_num.copy()
X_test_stand = X_test_num.copy()

# numerical features
num_cols = ['DepTime', 'ArrTime', 'AirTime', 'ArrDelay', 'DepDelay','Distance']

# apply standardization on numerical features
for i in num_cols:

    # fit on training data column
    scale = StandardScaler().fit(X_train_stand[[i]])
    
    # transform the training data column
    X_train_stand[i] = scale.transform(X_train_stand[[i]])
    
    # transform the testing data column
    X_test_stand[i] = scale.transform(X_test_stand[[i]])

X_train_stand.head()

In [ ]:
#import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (8, 12)
fig, axes = plt.subplots(3, 1)
sns.boxplot(data = X_train, orient='v' , ax=axes[0]).set(title='Original Data')
sns.boxplot(data = X_train_norm, orient='v' , ax=axes[1]).set(title='Normalized Data')
sns.boxplot(data = X_train_stand, orient='v' , ax=axes[2]).set(title='Standardized Data')
# move overall title up
fig.subplots_adjust(hspace = 0.2)

# add overall title
fig.suptitle('Comparison Data', x=0.5, y=0.93, fontsize=14, fontweight="bold")
plt.show()

In [ ]:
# Checking for the labels in the categorical parameters
print("There are {} Airlines".format(len(X_train['AL'].unique())))

In [ ]:
# Checking for the label counts in the categorical parameters
print("Each airline has the following number of entries :")
X_train['AL'].value_counts()

In [ ]:
print("Each airline has the following number of entries in percentage (%):")
X_train.AL.value_counts(normalize=True)

In [ ]:
p = X_train.AL.value_counts(normalize=True)[:15].sum().round(2)
print("The 15 airlines with the most registrations represent {} % of the total.:".format(p*100))

In [ ]:
print("The list of the top 15 airlines is as follows:")
top_15 = list(X_train.AL.value_counts(normalize=True)[:15].index)
print(*top_15, sep = ", ")

In [ ]:
# We create a mask for filtering airlines
mask = X_train['AL'].isin(top_15)

In [ ]:
# Create a new subset of data
subset_AL = X_train.loc[:, ['AL']]
subset_AL.head()

In [ ]:
# We created a new group with airlines that do not belong to the group of 15 airlines
subset_AL.loc[~mask,'AL'] = 'Others'
subset_AL.head()

In [ ]:
# We make a copy of the data frame
set_df = X_train.copy()

In [ ]:
# We label a new airline group "Other LA" with all airlines other than the 15 listed.
set_df.loc[~mask,'AL'] = 'Others'
set_df.head()

In [ ]:
# we group the records by airlines
grp = set_df.groupby(['AL']).sum()
grp.head(16)

In [ ]:
# we grouped the departure delay records by airlines and calculated their average value and the number of records
set2_df = set_df[['AL','ArrDelay']].groupby('AL').agg(['mean','count'])\
    .sort_values(by=('ArrDelay','mean'), ascending=False).round(2)
set2_df

In [ ]:
# we remove the multiple index and rename the columns
set2_df.columns =["Mean","Count"]
set2_df

In [ ]:
# we restart the index
set2_df = set2_df.reset_index()
set2_df

In [ ]:
import matplotlib.colors as pltc
from random import sample
# Defining colors
all_colors = [k for k,v in pltc.cnames.items()]
colors = sample(all_colors, 16)#len(global_stats)
print('colors: ', colors)

In [ ]:
# Creating dataset
cars = set2_df.AL.values
data = set2_df.Mean.values

# Creating explode data
explode = (0.1, 0.0, 0.2, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
#np.set_printoptions(precision = 1, suppress = True)
#explode = np.random.uniform(0,0.3,16)
#explode = tuple(explode)

# Creating color parameters
colors = tuple(colors)
 
# Wedge properties
wp = {'linewidth': 1, 'edgecolor': "green"}
 
# Creating autopct arguments
def func(pct, allvalues):
    absolute = int(pct / 100.*np.sum(allvalues))
    return "{:.1f}%\n({:d} g)".format(pct, absolute)

# Creating plot
fig, ax = plt.subplots(figsize=(10, 7))
wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data), explode=explode, labels=cars, \
    shadow=True, colors=colors, startangle=90, wedgeprops=wp, textprops=dict(color="black"))
 
# Adding legend
ax.legend(wedges, set2_df.AL, title="Airlines", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
plt.setp(autotexts, size=8, weight="bold")
ax.set_title("Arrival Delay")
 
# Show plot
plt.show()

In [ ]:
#  plot the mean of the records by airline
plt.figure(figsize=(12,8))
sns.barplot(x = "AL", y = "Mean", data = set2_df)
plt.show()

In [ ]:
# plot the number of records by airline 
plt.figure(figsize=(12,8))
sns.barplot(x = "AL", y = "Count", data = set2_df)
plt.show()

In [ ]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

In [ ]:
# droping the country column 
set_df = set_df.drop(['DepTime','ArrTime','AirTime','ArrDelay','DepDelay'], axis=1) 

In [ ]:
encoded_data = pd.get_dummies(set_df['AL'])
print(round(encoded_data, 1))

**Nivell 3**

- Exercici 3:
  
Resumeix les noves columnes generades de manera estadística i gràfica

In [ ]:
encoded_data.describe(include='all')

## Challenges of One-Hot Encoding: Dummy Variable Trap

One-Hot Encoding results in a Dummy Variable Trap as the outcome of one variable can easily be predicted with the help of the remaining variables.

> **Dummy Variable Trap is a scenario in which variables are highly correlated to each other.**

The Dummy Variable Trap leads to the problem known as **multicollinearity**. Multicollinearity occurs where there is a dependency between the independent features. Multicollinearity is a serious issue in machine learning models like [Linear Regression](https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/?utm_source=blog&utm_medium=one-hot-encoding-vs-label-encoding-using-scikit-learn) and [Logistic Regression](https://www.analyticsvidhya.com/blog/2015/11/beginners-guide-on-logistic-regression-in-r/?utm_source=blog&utm_medium=one-hot-encoding-vs-label-encoding-using-scikit-learn).

So, in order to overcome the problem of multicollinearity, one of the dummy variables has to be dropped. Here, I will practically demonstrate how the problem of multicollinearity is introduced after carrying out the one-hot encoding.

One of the common ways to check for multicollinearity is the Variance Inflation Factor (VIF):

* VIF=1, Very Less Multicollinearity
* VIF<5, Moderate Multicollinearity
* VIF>5, Extreme Multicollinearity (This is what we have to avoid)

Compute the VIF scores:

In [ ]:
import statsmodels.api as sm
# Function to calculate VIF
def calculate_vif(data):
    vif_df = pd.DataFrame(columns = ['Var', 'Vif'])
    x_var_names = data.columns
    for i in range(0, x_var_names.shape[0]):
        y = data[x_var_names[i]]
        x = data[x_var_names.drop([x_var_names[i]])]
        r_squared = sm.OLS(y,x).fit().rsquared
        vif = round(1/(1-r_squared),2)
        vif_df.loc[i] = [x_var_names[i], round(vif, 1)]
    return vif_df.sort_values(by = 'Vif', axis = 0, ascending=False, inplace=False)

df = encoded_data.copy()
calculate_vif(df)

In [ ]:
df2 = encoded_data.copy()
from statsmodels.stats.outliers_influence import variance_inflation_factor

#print(round(df2, 1))
# For each X, calculate VIF and save in dataframe
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df2.values, i) for i in range(df2.shape[1])]
vif["features"] = df2.columns
vif.round(1)

## Variance Inflation Factor (VIF) Explained

Colinearity is the state where two variables are highly correlated and contain similiar information about the variance within a given dataset. To detect colinearity among variables, simply create a correlation matrix and find variables with large absolute values. In R use the corr function and in python this can by accomplished by using numpy's corrcoef function.

Multicolinearity on the other hand is more troublesome to detect because it emerges when three or more variables, which are highly correlated, are included within a model. To make matters worst multicolinearity can emerge even when isolated pairs of variables are not colinear.

A common R function used for testing regression assumptions and specifically multicolinearity is ``"VIF()"`` and unlike many statistical concepts, its formula is straightforward:

$$
\text{V.I.F.} = \frac{1}{1-R^2}
$$

In [ ]:
X_train_num.describe()

In [ ]:
X_train_num.hist(figsize=(20, 12), color='#32a850', bins=24)
plt.show()

### The applied power transformation has transformed the features generating a standardized distribution of mean = 0 and std = 1 with different minimum and maximum values for each feature.

In [ ]:
X_train_num_power.describe()

In [ ]:
X_train_num_power.hist(figsize=(20, 12), color='#32a850', bins=24)
plt.show()

### The application of a quantile-type transformation with respect to the original distribution makes it look more like a Gaussian distribution. In cases where a bias has been detected, if it has not cancelled it out in the worst case it is possible to see a decrease in the bias.  

In [ ]:
X_train_num_quant.describe()

In [ ]:
X_train_num_quant.hist(figsize=(20, 12), color='#32a850', bins=24)
plt.show()

In [ ]:
X_train_time_dist.describe()

In [ ]:
X_train_time_dist.hist(figsize=(20, 12), color='#32a850', bins=24)
plt.show()